In [1]:
import numpy as np
import pygame
import sys
import math
import numpy as np
from mygrad import sliding_window_view
from collections import Counter
from time import time

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:


BLUE = (0,0,255)
BLACK = (0,0,0)
RED = (255,0,0)
YELLOW = (255,255,0)

RED_PLAYER = 1
YELLOW_PLAYER = 10

ROW_COUNT = 6
COLUMN_COUNT = 7


SQUARESIZE = 100
RADIUS = int(SQUARESIZE/2 - 5)
width = COLUMN_COUNT * SQUARESIZE
height = (ROW_COUNT+1) * SQUARESIZE

size = (width, height)

In [3]:
import pickle
with open('plays.pkl',mode='rb') as f:
    plays = pickle.load(f)
with open('wins.pkl',mode='rb') as f:
    wins = pickle.load(f)

In [4]:
def possible_fours(board):
    h = sliding_window_view(board,(1,4),1).reshape(-1,4)
    v = sliding_window_view(board.T,(1,4),1).reshape(-1,4)
    d1 = sliding_window_view(board,(4,4),1).reshape(-1,4,4).transpose(1,2,0).diagonal()
    d2 = sliding_window_view(board[:,::-1],(4,4),1).reshape(-1,4,4).transpose(1,2,0).diagonal()
    return h,v,d1,d2

def four_in_row(board):
    for possible_four in possible_fours(board):
        total = np.sum(possible_four,axis=1)
        if total[total == RED_PLAYER * 4].size > 0:

            return RED_PLAYER
        if total[total == YELLOW_PLAYER * 4].size > 0:
            return YELLOW_PLAYER
    if 0 not in board:
        return -1
    return 0
def valid_move(move,column_heights):
    moves = possible_moves(column_heights)
    return moves[moves == move].size
def possible_moves(column_heights):
    return np.array(np.where(column_heights < 6)).reshape(-1)
def apply_move(move,color,board,column_heights):
    assert  column_heights[move] < 6
    
    board[int(5-column_heights[move]),move] = color
    column_heights[move] += 1
    return board

def next_boards(color,board,column_heights):
    next = []
    for move in possible_moves(column_heights):
        next.append(str(apply_move(move,color,board.copy(),column_heights.copy())))
    return next
def change_color(color):
    return RED_PLAYER if color == YELLOW_PLAYER else YELLOW_PLAYER
def get_MCTS_move(board,columm_heights,color,calc_time=5,plays=None,wins=None,stats=True):
    
    moves = possible_moves(columm_heights)
    if moves.size == 0:
        return
    if moves.size == 1:
        return moves[0]
    if plays is None:
        plays = Counter()
    if wins is None:
        wins = Counter()
    
    games = 0
    begin = time()
    while time() - begin < calc_time:
        b = board.copy()
        c = columm_heights.copy()
        run_MCTS_sim(b,c,color,plays,wins)
        games += 1

    winrate, move = max(
            (wins.get((color, S), 0) /
             plays.get((color, S), 1),
             p)
            for p, S in zip(moves,next_boards(color,board,columm_heights)))
    if stats:
        print("simulations:{} move:{} winrate:{}".format(games,move,winrate))
    return move
        
def run_MCTS_sim(board,column_heights,color,plays,wins):
    visited_boards = set()
    turn = np.sum(column_heights)
    win = 0
    expand = 2
    for t in np.arange(42 - turn):
        legal_moves = possible_moves(column_heights)
        move = np.random.choice(legal_moves)
        apply_move(move,color,board,column_heights)
        if expand and (color, str(board)) not in plays:
            expand -= 1
            plays[(color,str(board))] = 0
            wins[(color,str(board))] = 0
        visited_boards.add((color,str(board)))
        
        color = change_color(color)
        
        win = four_in_row(board)
        if win:
            break
    for c, b in visited_boards:
        if (c,b) not in plays:
            continue
        plays[(c,b)] += 1
        if c == win:
            wins[(c,b)] += 1
    return plays, wins
            
        
    winrate, move = max(
            (wins.get((color, S), 0) /
             plays.get((color, S), 1),
             p)
            for p, S in zip(moves,next_boards(color,board,columm_heights)))
    if stats:
        print("simulations:{} move:{} winrate:{}".format(games,move,winrate))
    return move
def draw_board(board,screen):
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, BLUE, (c*SQUARESIZE, r*SQUARESIZE+SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, BLACK, (int(c*SQUARESIZE+SQUARESIZE/2), int(r*SQUARESIZE+SQUARESIZE+SQUARESIZE/2)), RADIUS)

    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if board[r][c] == RED_PLAYER:
                pygame.draw.circle(screen, RED, (int(c*SQUARESIZE+SQUARESIZE/2), int((1+r)*SQUARESIZE+SQUARESIZE/2)), RADIUS)
            elif board[r][c] == YELLOW_PLAYER: 
                pygame.draw.circle(screen, YELLOW, (int(c*SQUARESIZE+SQUARESIZE/2), int((1+r)*SQUARESIZE+SQUARESIZE/2)), RADIUS)
    pygame.display.update()

In [5]:

def play(player1,player2,plays=Counter(),wins=Counter()):
    board = np.zeros((6,7),dtype=np.int)
    column_heights = np.zeros(7,dtype=np.int)
    pygame.init()
    screen = pygame.display.set_mode(size)
    
    draw_board(board,screen)
    pygame.display.update()

    myfont = pygame.font.SysFont("monospace", 75)
    color = RED_PLAYER
    game_over = False
    while not game_over:
        if color == RED_PLAYER and player1 == 'computer':
            move = get_MCTS_move(board,column_heights,color,plays=plays,wins=wins)
            apply_move(move,color,board,column_heights)
            color = change_color(color)
                
        if color == YELLOW_PLAYER and player2 == 'computer':
            
            move = get_MCTS_move(board,column_heights,color,plays=plays,wins=wins)
            apply_move(move,color,board,column_heights)
            color = change_color(color)
        else:    
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    sys.exit()

                if event.type == pygame.MOUSEMOTION:
                    pygame.draw.rect(screen, BLACK, (0,0, width, SQUARESIZE))
                    posx = event.pos[0]
                    if color  == RED_PLAYER:
                        pygame.draw.circle(screen, RED, (posx, int(SQUARESIZE/2)), RADIUS)
                    else: 
                        pygame.draw.circle(screen, YELLOW, (posx, int(SQUARESIZE/2)), RADIUS)
                pygame.display.update()

                if event.type == pygame.MOUSEBUTTONDOWN:
                    pygame.draw.rect(screen, BLACK, (0,0, width, SQUARESIZE))

                    # Ask for Player 1 Input
                    if color  == RED_PLAYER:
                        if (player1 == 'human'):
                            posx = event.pos[0]
                            move = int(math.floor(posx/SQUARESIZE))


                            if valid_move(move,column_heights):
                                apply_move(move,color,board,column_heights)
                                color = change_color(color)


                    # # Ask for Player 2 Input
                    else:

                        if (player2 == 'human'):
                            posx = event.pos[0]
                            move = int(math.floor(posx/SQUARESIZE))
                            if valid_move(move,column_heights):
                                apply_move(move,color,board,column_heights) 
                                color = change_color(color)



        game_over = four_in_row(board)        
        if game_over == RED_PLAYER: 
            label = myfont.render("Red wins!!", 1, RED)
            screen.blit(label, (40,10))
        if game_over == YELLOW_PLAYER: 
            label = myfont.render("Yellow wins!!", 1, YELLOW)
            screen.blit(label, (40,10))
        if game_over == -1: 
            label = myfont.render("DRAW", 1, BLUE)
            screen.blit(label, (40,10))

        draw_board(board,screen)
        
        

        if game_over:
            pygame.time.wait(3000)
            sys.exit()             

In [6]:

def computer_vs_computer(plays=Counter(),wins=Counter()):
    board = np.zeros((6,7),dtype=np.int)
    column_heights = np.zeros(7,dtype=np.int)
    pygame.init()
    screen = pygame.display.set_mode(size)
    
    draw_board(board,screen)
    pygame.display.update()

    myfont = pygame.font.SysFont("monospace", 75)
    color = RED_PLAYER
    game_over = False
    while not game_over:
        if color == RED_PLAYER:
            move = get_MCTS_move(board,column_heights,color,plays=plays,wins=wins)
            apply_move(move,color,board,column_heights)
            color = change_color(color)
                
        if color == YELLOW_PLAYER:
            
            move = get_MCTS_move(board,column_heights,color,plays=plays,wins=wins)
            apply_move(move,color,board,column_heights)
            color = change_color(color)
        game_over = four_in_row(board)        
        if game_over == RED_PLAYER: 
            label = myfont.render("Red wins!!", 1, RED)
            screen.blit(label, (40,10))
        if game_over == YELLOW_PLAYER: 
            label = myfont.render("Yellow wins!!", 1, YELLOW)
            screen.blit(label, (40,10))
        if game_over == -1: 
            label = myfont.render("DRAW", 1, BLUE)
            screen.blit(label, (40,10))

        draw_board(board,screen)
        
        

        if game_over:
            pygame.time.wait(3000)
            sys.exit()             

In [ ]:
play('human','computer',plays,wins)


In [8]:
import pickle
with open('plays.pkl',mode='wb') as f:
    pickle.dump(plays,f)
with open('wins.pkl',mode='wb') as f:
    pickle.dump(wins,f)